In [ ]:
import sys

In [ ]:
from google.colab import drive
drive.mount("mnt", force_remount=True)

In [ ]:
%cd "/content/mnt/My Drive/Data/train_data/BirdCLEF2020_Train_Audio/audio"

In [ ]:
path="/content/mnt/My Drive/Data/train_data/BirdCLEF2020_Train_Audio/audio/"


In [ ]:
!pip3 install import_ipynb
import import_ipynb

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline

In [ ]:
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['xtick.labelsize'] = 18
plt.rcParams['ytick.labelsize'] = 18

In [ ]:
 ## read in
file_path = 'Accipiter_cooperii_Coopers_Hawk/XC503265.mp3'
sig_orig, sr_orig = librosa.load(file_path,sr=None)

## resample
#sig_orig = librosa.resample(sig_orig, sr_orig, 44100)

## Mel power spectrogram
melspec = librosa.feature.melspectrogram(y=sig_orig, sr=sr_orig)

In [ ]:
print(sr_orig)
print(sig_orig)

In [ ]:
plt.figure(figsize=(12,7))
plt.imshow(melspec[:,:int(melspec.shape[1]/8.)], norm=LogNorm())
plt.xlabel('Time bins'); plt.ylabel('Frequency bins'); plt.colorbar(format='%+2.0f dB'); plt.title('Spectrogram');

In [ ]:
import librosa
import numpy as np
import matplotlib.pyplot as plt
import librosa.display


y, sr = librosa.load(file_path, sr=44100)

X = librosa.stft(y)
Xdb = librosa.amplitude_to_db(abs(X))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()
plt.figure(figsize=(14, 5))
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='log')
plt.colorbar()

plt.figure(figsize=(14, 5))
librosa.display.waveplot(y, sr=sr)
plt.figure(figsize=(14, 5))
# short time fourier transform
# (n_fft and hop length determine frequency/time resolution)
n_fft = 2048
S = librosa.stft(y, n_fft=n_fft, hop_length=n_fft//2)
# convert to db
D = librosa.amplitude_to_db(np.abs(S), ref=np.max)
# average over file
D_AVG = np.mean(D, axis=1)

plt.bar(np.arange(D_AVG.shape[0]), D_AVG)
x_ticks_positions = [n for n in range(0, n_fft // 2, n_fft // 16)]
x_ticks_labels = [str(sr / 2048 * n) + 'Hz' for n in x_ticks_positions]
plt.xticks(x_ticks_positions, x_ticks_labels)
plt.xticks(fontsize=14, rotation=90)
plt.xlabel('Frequency')
plt.ylabel('dB')
plt.show()

In [ ]:
import librosa
import numpy
y, sr = librosa.load(file_path)
n_fft = 1024
hop_length = 12

spec = numpy.abs(librosa.core.stft(y, n_fft=n_fft, hop_length=hop_length))
freqs = librosa.core.fft_frequencies(n_fft=n_fft)
times = librosa.core.frames_to_time(spec[0], sr=sr, n_fft=n_fft, hop_length=hop_length)

print('spectrogram size', spec.shape)

fft_bin = 100
time_idx = 1000

print('freq (Hz)', freqs[fft_bin])
print('time (s)', times[time_idx])
print('amplitude', spec[fft_bin, time_idx])

In [ ]:
#spectral centroid -- centre of mass -- weighted mean of the frequencies present in the sound
import sklearn
spectral_centroids = librosa.feature.spectral_centroid(y, sr=sr)[0]
spectral_centroids.shape
# Computing the time variable for visualization
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)
# Normalising the spectral centroid for visualisation
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)
#Plotting the Spectral Centroid along the waveform
librosa.display.waveplot(y, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='r')

In [ ]:
spectral_rolloff = librosa.feature.spectral_rolloff(y, sr=sr)[0]
librosa.display.waveplot(y, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_rolloff), color='r')

In [ ]:
mfccs = librosa.feature.mfcc(y, sr=sr)
print(mfccs.shape)
#Displaying  the MFCCs:
plt.figure(figsize=(15, 7))
librosa.display.specshow(mfccs, sr=sr, x_axis='time')

plt.colorbar()

#20 is my number of mfccs i.e. features and 393 samples

In [ ]:
spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(y+0.01, sr=sr)[0]
spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(y+0.01, sr=sr, p=3)[0]
spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(y+0.01, sr=sr, p=4)[0]
plt.figure(figsize=(15, 9))
librosa.display.waveplot(y, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_bandwidth_2), color='r')
plt.plot(t, normalize(spectral_bandwidth_3), color='g')
plt.plot(t, normalize(spectral_bandwidth_4), color='y')
plt.legend(('p = 2', 'p = 3', 'p = 4'))

In [ ]:
zero_crossings = librosa.zero_crossings(y[9000:9100], pad=False)
print(sum(zero_crossings))

In [ ]:
from scipy.signal import butter, lfilter


def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y
    

In [ ]:
from scipy.signal import freqz
lowcut = 5000.0
highcut =8000.0
fs=sr
plt.figure(1)
plt.clf()
for order in [3, 6, 9]:
  b, a = butter_bandpass(lowcut, highcut, fs, order=order)
  w, h = freqz(b, a, worN=2000)
  plt.plot((fs * 0.5 / np.pi) * w, abs(h), label="order = %d" % order)
#plot of my bandpass filter
plt.plot([0, 0.5 * fs], [np.sqrt(0.5), np.sqrt(0.5)],
             '--', label='sqrt(0.5)')
plt.figure()
filter_y= butter_bandpass_filter(y, lowcut, highcut, fs, order=6)
plt.plot(filter_y, label='Filtered signal')
plt.xlabel('time (seconds)')
plt.grid(True)
plt.axis('tight')
plt.legend(loc='upper left')
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
librosa.display.waveplot(filter_y, sr=sr)
plt.figure(figsize=(14, 5))
# short time fourier transform
# (n_fft and hop length determine frequency/time resolution)
n_fft = 2048
S = librosa.stft(filter_y, n_fft=n_fft, hop_length=n_fft//2)
# convert to db
D = librosa.amplitude_to_db(np.abs(S), ref=np.max)
# average over file
D_AVG = np.mean(D, axis=1)

plt.bar(np.arange(D_AVG.shape[0]), D_AVG)
x_ticks_positions = [n for n in range(0, n_fft // 2, n_fft // 16)]
x_ticks_labels = [str(sr / 2048 * n) + 'Hz' for n in x_ticks_positions]
plt.xticks(x_ticks_positions, x_ticks_labels)
plt.xticks(fontsize=14, rotation=90)
plt.xlabel('Frequency')
plt.ylabel('dB')
plt.show()

In [ ]:
import sklearn
spectral_centroids = librosa.feature.spectral_centroid(filter_y, sr=sr)[0]
# Computing the time variable for visualization
plt.figure(figsize=(12, 4))
frames = range(len(spectral_centroids))
t = librosa.frames_to_time(frames)
# Normalising the spectral centroid for visualisation
def normalize(x, axis=0):
    return sklearn.preprocessing.minmax_scale(x, axis=axis)
#Plotting the Spectral Centroid along the waveform
librosa.display.waveplot(filter_y, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_centroids), color='b')


In [ ]:
spectral_bandwidth_2 = librosa.feature.spectral_bandwidth(filter_y+0.01, sr=sr)[0]
spectral_bandwidth_3 = librosa.feature.spectral_bandwidth(filter_y+0.01, sr=sr, p=3)[0]
spectral_bandwidth_4 = librosa.feature.spectral_bandwidth(filter_y+0.01, sr=sr, p=4)[0]
plt.figure(figsize=(15, 9))
librosa.display.waveplot(y, sr=sr, alpha=0.4)
plt.plot(t, normalize(spectral_bandwidth_2), color='r')
plt.plot(t, normalize(spectral_bandwidth_3), color='g')
plt.plot(t, normalize(spectral_bandwidth_4), color='y')
plt.legend(('p = 2', 'p = 3', 'p = 4'))

In [ ]:
zero_crossings = librosa.zero_crossings(filter_y[9000:9100], pad=False)
print(sum(zero_crossings))


In [ ]:
mfccs = librosa.feature.mfcc(filter_y, sr=sr)
print(mfccs.shape)
#Displaying  the MFCCs:
plt.figure(figsize=(15, 7))
librosa.display.specshow(mfccs, sr=sr, x_axis='time')

In [ ]:
chromagram = librosa.feature.chroma_stft(filter_y, sr=sr, hop_length=9000)
plt.figure(figsize=(15, 5))
librosa.display.specshow(chromagram, x_axis='time', y_axis='chroma', hop_length=9000, cmap='coolwarm')

In [ ]:
import subprocess

# define the ls command
ls = subprocess.Popen(["ls", "-p", "."],
                      stdout=subprocess.PIPE,
                     )
# define the grep command
grep = subprocess.Popen(["grep", "-v", "$/"],
                        stdin=ls.stdout,
                        stdout=subprocess.PIPE,
                        )

# read from the end of the pipe (stdout)
endOfPipe = grep.stdout
classlist=[]
# output the files line by line
for line in endOfPipe:
    classlist.append(line.decode('ascii').rstrip("\n"))
print(classlist)

In [ ]:
import os

i=0
output_path='/content/mnt/My Drive/Filtered Sound/'
if not os.path.exists(output_path):
  print("Path not present Creating now")
  os.mkdir(output_path)
files_class=dict()
for folder in classlist:
  files=subprocess.Popen(["ls", path+folder],
                        stdout=subprocess.PIPE,
                         )
  # read from the end of the pipe (stdout)
  endOfPipe = files.stdout
  # output the files line by line
  print(path+folder)
  if not os.path.exists(output_path+folder):
    os.mkdir(output_path+folder)
  print("In Folder", folder)

  for line in endOfPipe:
    print("   Creating file",line.decode('ascii').rstrip("\n") )
    y, sr = librosa.load(path+folder+line.decode('ascii').rstrip("\n"), sr=44100)
    filter_y= butter_bandpass_filter(y, lowcut, highcut, fs, order=6)
    librosa.output.write_wav(output_path+folder+line.decode('ascii').rstrip("\n"), filter_y, sr)      
  if i==10:
    break
  i=i+1